In [14]:
import glob
import numpy as np
import pandas as pd
from datetime import datetime, date
import os
import matplotlib.pyplot as plt
import plotly.express as px

from talib import RSI, BBANDS, MACD

import talib

In [15]:
data_path =  os.path.join(os.getcwd(),"nse_data\\data\\")
data_index = os.path.join(os.getcwd(),"nse_data\\index_data\\")
dump_path =  os.path.join(os.getcwd(),"nse_data\\dump\\")
meta_data =  os.path.join(os.getcwd(),"nse_data\\meta_data\\")
reports_path = os.path.join(os.getcwd(),"nse_data\\reports\\")
nse_indices = os.path.join(os.getcwd(),"nse_data\\nse_indices\\")
# print(nse_indices)

In [16]:
nifty500_df = pd.read_csv(os.path.join(nse_indices, "ind_nifty500list.csv"))
stocks_list = nifty500_df.Symbol.tolist()
len(stocks_list)

501

In [17]:
stocks_list = list(dict.fromkeys(stocks_list))

add_stocks = ['VEDL', 'YESBANK', 'SBICARD']
skip_stocks = ['BBTC','IRB','M&MFIN','PVR']

stocks_list = stocks_list + add_stocks
stocks_list = sorted(set(stocks_list)-(set(skip_stocks)))
len(stocks_list)

# stocks_list = random.sample(stocks_list, len(stocks_list))

499

In [29]:
start_date = '2020-08-01'
end_date = date.today()

In [30]:
# Function to get script data file path if present
def get_stock_file_path(script):
    for file in glob.glob(data_path + script + '_*.csv'):
        file = file.replace("\\","/")
        return(file)
    
def get_stock_filename(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)
    
# Function to check if the script data file already present or not: True or False
def stock_data_present(script):
    file = get_stock_file_path(script)
    if file:
        return True
    return False

# Function to get stock dataframe from script name
def stock_df_from_csv(script):
    file = get_stock_file_path(script)
    stock_df = pd.read_csv(file)
    return stock_df

# Function to get stock dataframe from script name with Date as Index
def get_df_date_index(stock):
    df = stock_df_from_csv(stock)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', drop=True, inplace = True)
    df.index.name = None
    return df


def calculate_macd(df,  field = 'Close' ):
    
    macd, macdsignal, macdhist = MACD(df[field], fastperiod=12, slowperiod=26, signalperiod=9)
    macd.name, macdsignal.name, macdhist.name = 'macd','macdsignal','macdhist'
    macd, macdsignal, macdhist = macd.to_frame(), macdsignal.to_frame(), macdhist.to_frame()

    df = df.merge(macd, left_index=True, right_index=True)
    df = df.merge(macdsignal, left_index=True, right_index=True)
    df = df.merge(macdhist, left_index=True, right_index=True)
    return df

In [ ]:
# Function to create MACD Report

def macd_report(stocks):
    macd_df = []
    for stock in stocks:
        df = get_df_date_index(stock)
        df = calculate_macd(df)        
        df = df[start_date:end_date]
        df = df[["Close","macd","macdsignal", "macdhist"]]
        df = np.round(df, decimals=4)
        df.index.name ="Date"
        df = df.reset_index()
#         display(df)
        for ind in df.index: 
#             print(ind-1)
            if ind+1 < df.shape[0]:
                if (df['ema9'][ind] < df['ema26'][ind] ) & (df['ema9'][ind+1] > df['ema26'][ind+1]):
#                     print(stock ,df['Date'][ind] , df['sma50'][ind] , df['sma200'][ind] , df['sma50'][ind+1] , df['sma200'][ind+1] , sep = "\t\t")
                    ema_cross_df.append(
                                {   'Stock': stock,
                                    'Date': df['Date'][ind],
                                    'EMA9':df['ema9'][ind],
                                    'EMA26':df['ema26'][ind]
                                })
                    ema_cross_df.append(
                                {   'Stock': stock,
                                    'Date': df['Date'][ind+1],
                                    'EMA9':df['ema9'][ind+1],
                                    'EMA26':df['ema26'][ind+1]
                                }
                        
                            )
    return pd.DataFrame(ema_cross_df)

   